In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn import metrics
import xgboost as xgb
import tensorflow as tf
from gc import collect

In [3]:
df_data=pd.read_csv("DataSetNormalised.csv")
df_label=pd.read_csv("LabelSet.csv")

In [4]:
x=np.array(df_data.iloc[:,1:])
y=np.array(df_label.iloc[:,1])
x=x.astype(np.float32)
x_test=x[10000000:11000000][:]
y_test=y[10000000:11000000][:]
x_train=x[:10000000][:]
y_train=y[:10000000][:]

In [119]:
del df_data
del df_label
del x
del y
collect()

1397

In [5]:
model_xgb=XGBClassifier()
model_xgb.load_model("XGBoost.json")
y_test_pred_xgb=model_xgb.predict_proba(x_test)
y_train_pred_xgb=model_xgb.predict_proba(x_train)
y_pred_xgb=y_test_pred_xgb[:,1]
y_pred_xgb_train=y_train_pred_xgb[:,1]

In [6]:
y_test_pred_xgb=np.reshape(y_pred_xgb,(1000000,1))
y_train_pred_xgb=np.reshape(y_pred_xgb_train,(10000000,1))
y_train_pred_xgb.shape

(10000000, 1)

In [1]:
model_DNN_5 = tf.keras.models.load_model('DNN_5')
y_test_pred_DNN_5=model_DNN_5.predict(x_test)
y_train_pred_DNN_5=model_DNN_5(x_train)

NameError: name 'tf' is not defined

In [ ]:
model_NN_Shallow_1024 = tf.keras.models.load_model('NN_Shallow_1024')
y_test_pred_NN_Shallow_1024=model_NN_Shallow_1024.predict(x_test)
y_train_pred_NN_Shallow_1024=model_NN_Shallow_1024.predict(x_train)

In [ ]:
model_NN_Shallow_2048 = tf.keras.models.load_model('NN_Shallow_2048')
y_test_pred_NN_Shallow_2048=model_NN_Shallow_2048.predict(x_test)
y_train_pred_NN_Shallow_2048=model_NN_Shallow_2048.predict(x_train)

In [76]:
model_NN_Shallow_10240 = tf.keras.models.load_model('NN_Shallow_10240')
y_test_pred_NN_Shallow_10240=model_NN_Shallow_10240.predict(x_test)
y_test_pred_NN_Shallow_10240=model_NN_Shallow_10240.predict(x_train)

The training auc 0.8140237759861899


In [77]:
model_ResNet_52 = tf.keras.models.load_model('ResNet_52')
y_test_pred_model_ResNet_52=model_ResNet_52.predict(x_test)
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_model_ResNet_52)
auc=metrics.auc(fpr, tpr)
print("The training auc",str(auc))

The training auc 0.8693254776013825


In [116]:
y_pred_ensemble=y_test_pred_DNN_5
y_pred_ensemble=y_pred_ensemble+y_test_pred_model_ResNet_52
y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_1024
y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_10240
y_pred_ensemble=y_pred_ensemble+y_test_pred_NN_Shallow_2048
y_pred_ensemble=y_pred_ensemble+y_test_pred_xgb
y_pred_ensemble=y_pred_ensemble/5
y_pred_ensemble.shape
fpr, tpr, thresholds = roc_curve(y_test, y_pred_ensemble)
metrics.auc(fpr, tpr)
#print("The training accuracy",str(accuracy_score(y_test, np.round(y_pred_ensemble))))

0.8759870002247364

In [24]:
#Trying out average ensemble
y_pred_ensemble=y_test_pred_DNN_5+0*y_test_pred_NN_Shallow_1024
y_pred_ensemble=np.round(y_pred_ensemble)

accuracy = accuracy_score(y_test, y_pred_ensemble)
print("The training accuracy",str(accuracy))
fpr, tpr, thresholds = roc_curve(y_test, y_pred_ensemble)
metrics.auc(fpr, tpr)


The training accuracy 0.788488


0.7865833286210102

In [123]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()